imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="FreezeVsTune"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    "numOfTrials": 5
}

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix data splits

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        # update experiment params
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        # fix data split names
        old_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params))
        old_datasplitsPath = os.path.join(experimentsPath, experimentName, old_dataset_name)
        new_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params_new))
        new_datasplitsPath = os.path.join(experimentsPath, experimentName, new_dataset_name)
        
        try:
            if os.path.exists(old_datasplitsPath):

                # update the folder names
                os.rename(old_datasplitsPath, new_datasplitsPath)
                print("datasplit", old_datasplitsPath, '->', new_datasplitsPath, "updated")

                # update the params.json
                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_datasplitsPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
                
            else:
                raise
                
        except:
            print("datasplit", old_dataset_name, "could not be fixed to", new_dataset_name)
            pass   
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['datasetName'] = new_dataset_name

        
        bar.update()

datasplit datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a could not be fixed to datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a
datasplit datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a could not be fixed to datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a



Fix models

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/FreezeVsTune/models/8fae45a05ce6694bacd44407373668385e44c9db5631acb8bdf5a698 -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/8fae45a05ce6694bacd44407373668385e44c9db5631acb8bdf5a698 updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/models/8fae45a05ce6694bacd44407373668385e44c9db5631acb8bdf5a698/params.json updated
model /home/elhamod/HGNN/experiments/FreezeVsTune/models/3fdef5ba2f52e82a736f0eae926f02363e557efcd0fa7dd2dce966ae -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/3fdef5ba2f52e82a736f0eae926f02363e557efcd0fa7dd2dce966ae updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/models/3fdef5ba2f52e82a736f0eae926f02363e557efcd0fa7dd2dce966ae/params.json updated
model /home/elhamod/HGNN/experiments/FreezeVsTune/models/9fba5b6d5bf04393a7ea39fd456096338098f147943bd1aa66982d25 -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/9fba5b6d5bf04393a7ea39fd456096338098f147943bd1aa66982d25 updated
json /home/elhamod/HGNN/experiments

model /home/elhamod/HGNN/experiments/FreezeVsTune/models/21732080ae91025dd7678a0c34910ff99bd6ebe781a48abb050fd418 -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/21732080ae91025dd7678a0c34910ff99bd6ebe781a48abb050fd418 updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/models/21732080ae91025dd7678a0c34910ff99bd6ebe781a48abb050fd418/params.json updated
model /home/elhamod/HGNN/experiments/FreezeVsTune/models/f21a7bdcdd8620f78796e22ed744dea8df46b3ec98297daed3fedff4 -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/f21a7bdcdd8620f78796e22ed744dea8df46b3ec98297daed3fedff4 updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/models/f21a7bdcdd8620f78796e22ed744dea8df46b3ec98297daed3fedff4/params.json updated
model /home/elhamod/HGNN/experiments/FreezeVsTune/models/5b7515330237c2852d135f2ce663ae870eca9e163fdcc7b273d41757 -> /home/elhamod/HGNN/experiments/FreezeVsTune/models/5b7515330237c2852d135f2ce663ae870eca9e163fdcc7b273d41757 updated
json /home/elhamod/HGNN/experiments

Fix results directory

In [7]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result /home/elhamod/HGNN/experiments/FreezeVsTune/results/d6873ee3953c93b89c9092ab603ce1dc8242e83b8495fd63007b1cc9 -> /home/elhamod/HGNN/experiments/FreezeVsTune/results/d6873ee3953c93b89c9092ab603ce1dc8242e83b8495fd63007b1cc9 updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/results/d6873ee3953c93b89c9092ab603ce1dc8242e83b8495fd63007b1cc9/params.json updated
result /home/elhamod/HGNN/experiments/FreezeVsTune/results/1e1fb23ee91b9ce31e8f09e5635e8950c1fe5ba74983b13f7d6483bc -> /home/elhamod/HGNN/experiments/FreezeVsTune/results/1e1fb23ee91b9ce31e8f09e5635e8950c1fe5ba74983b13f7d6483bc updated
json /home/elhamod/HGNN/experiments/FreezeVsTune/results/1e1fb23ee91b9ce31e8f09e5635e8950c1fe5ba74983b13f7d6483bc/params.json updated



Fix experiments file

In [8]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash 8fae45a05ce6694bacd44407373668385e44c9db5631acb8bdf5a698 -> 8fae45a05ce6694bacd44407373668385e44c9db5631acb8bdf5a698 updated
trialHash 3fdef5ba2f52e82a736f0eae926f02363e557efcd0fa7dd2dce966ae -> 3fdef5ba2f52e82a736f0eae926f02363e557efcd0fa7dd2dce966ae updated
trialHash 9fba5b6d5bf04393a7ea39fd456096338098f147943bd1aa66982d25 -> 9fba5b6d5bf04393a7ea39fd456096338098f147943bd1aa66982d25 updated
trialHash 7761bb0425454f6523cdee932958381b8e706cd632f4ddd9dda30001 -> 7761bb0425454f6523cdee932958381b8e706cd632f4ddd9dda30001 updated
trialHash 0ac8ef08f232baf463b197b7fc76a3378d8e16c7ff1820158c867937 -> 0ac8ef08f232baf463b197b7fc76a3378d8e16c7ff1820158c867937 updated



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


trialHash 21732080ae91025dd7678a0c34910ff99bd6ebe781a48abb050fd418 -> 21732080ae91025dd7678a0c34910ff99bd6ebe781a48abb050fd418 updated
trialHash f21a7bdcdd8620f78796e22ed744dea8df46b3ec98297daed3fedff4 -> f21a7bdcdd8620f78796e22ed744dea8df46b3ec98297daed3fedff4 updated
trialHash 5b7515330237c2852d135f2ce663ae870eca9e163fdcc7b273d41757 -> 5b7515330237c2852d135f2ce663ae870eca9e163fdcc7b273d41757 updated
trialHash 50ec050432a5d682ee0df329034c0d1f9bc6329e1f105d70f6834b30 -> 50ec050432a5d682ee0df329034c0d1f9bc6329e1f105d70f6834b30 updated
trialHash 1533bedb096e38cd6daf05b062b368f5c547cff4c97be6f9002d9798 -> 1533bedb096e38cd6daf05b062b368f5c547cff4c97be6f9002d9798 updated




Fix params file

In [9]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")


json /home/elhamod/HGNN/experiments/FreezeVsTune/params.json updated
